## Limpieza rapida de los archivos Shapefile con limites territoriales tanto de los municipios(colonias) y las alcaldias

In [43]:
import geopandas as gpd
import re
import pandas as pd
from copy_repo_funciones import *
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
geo= gpd.read_file('../FINAL/RAW/GEOREF/georef-mexico-colonia/georef-mexico-colonia-millesime.shp', encoding='utf-8')

In [14]:
geo.head()

,year,sta_code,sta_name,mun_code,mun_name,col_code,col_name,col_area_co,col_type,geometry
0,2022,09,Ciudad de México,09002,Azcapotzalco,02-009,Coltongo,MEX,colonia,"POLYGON ((-99.15595 19.48651, -99.15294 19.485..."
1,2022,09,Ciudad de México,09003,Coyoacán,03-006,Ampliacion Candelaria,MEX,colonia,"POLYGON ((-99.15265 19.32581, -99.15336 19.326..."
2,2022,09,Ciudad de México,09003,Coyoacán,03-026,Croc Culhuacan Secc 6 (U Hab),MEX,colonia,"POLYGON ((-99.12251 19.32904, -99.12198 19.329..."
3,2022,09,Ciudad de México,09003,Coyoacán,03-072,Los Cipreses,MEX,colonia,"POLYGON ((-99.12466 19.31520, -99.12491 19.315..."
4,2022,09,Ciudad de México,09004,Cuajimalpa de Morelos,04-033,Lomas Del Chamizal,MEX,colonia,"POLYGON ((-99.26654 19.38314, -99.26658 19.383..."


In [10]:
geo['col_name'] = geo['col_name'].str.replace(r"[\[\]']", '', regex=True)
geo['sta_code'] = geo['sta_code'].str.replace(r"[\[\]']", '', regex=True)
geo['sta_namee'] = geo['sta_name'].str.replace(r"[\[\]']", '', regex=True)

In [13]:
geo = geo.apply(lambda x: x.str.replace(r"[\[\]']", '', regex=True) if x.dtype == 'object' else x)

In [16]:
geo.to_file('../FINAL/modified_geo.shp')

C:\Users\anton\AppData\Local\Temp\ipykernel_11828\3943655620.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  geo.to_file('../FINAL/modified_geo.shp')


### TANDEO

A Tandeo hay que agregarle ID de la colonia para las relaciones cardinales en Tableau

In [29]:
tandeo=gpd.read_file('../CLEAN/TABLEAU/tandeo2019.shp', encoding='utf-8')

In [32]:
tandeo.head()

,ID,ENTIDAD,MUNICIPIO,NOMBRE,DELEGACIO,tnd_2019,geometry
0,2059,9,13,DIF,Xochimilco,no,"POLYGON ((-99.09734 19.24389, -99.09614 19.243..."
1,1,9,2,SAN ALVARO,Azcapotzalco,no,"POLYGON ((-99.18803 19.46367, -99.18668 19.463..."
2,1033,9,12,XITLE,Tlalpan,no,"POLYGON ((-99.18350 19.25851, -99.18314 19.258..."
3,2060,9,13,SAN JUAN DE LOS LAGOS(TEZONTITLA),Xochimilco,no,"POLYGON ((-99.11681 19.21855, -99.11656 19.218..."
4,2,9,2,SAN SALVADOR XOCHIMANCA,Azcapotzalco,no,"POLYGON ((-99.17763 19.46195, -99.17699 19.461..."


In [31]:
tandeo['DELEGACIO'] = tandeo['DELEGACIO'].str.title()

In [33]:
tandeo.loc[tandeo['DELEGACIO']=='Coyoacan', 'DELEGACIO']='Coyoacán'
tandeo.loc[tandeo['DELEGACIO']=='Alvaro Obregon', 'DELEGACIO']='Álvaro Obregón'
tandeo.loc[tandeo['DELEGACIO']=='Tlahuac', 'DELEGACIO']='Tláhuac'
tandeo.loc[tandeo['DELEGACIO']=='Benito Juarez', 'DELEGACIO']='Benito Juárez'
tandeo.loc[tandeo['DELEGACIO'] == 'Cuauhtemoc', 'DELEGACIO'] = 'Cuauhtémoc'
tandeo.loc[tandeo['DELEGACIO'] == 'Magdalena Contreras', 'DELEGACIO'] = 'La Magdalena Contreras'

In [47]:
tandeo['DELEGACIO'].value_counts()

DELEGACIO
Iztapalapa                375
Álvaro Obregón            313
Tlalpan                   300
Gustavo A. Madero         281
Tláhuac                   220
Xochimilco                195
Coyoacán                  142
Azcapotzalco              111
Venustiano Carranza        91
Miguel Hidalgo             80
Cuajimalpa de Morelos      65
Benito Juárez              57
La Magdalena Contreras     51
Iztacalco                  47
Milpa Alta                 38
Cuauhtémoc                 33
Name: count, dtype: int64

In [46]:
tandeo.loc[tandeo['DELEGACIO']=='Cuajimalpa De Morelos', 'DELEGACIO']='Cuajimalpa de Morelos'

In [48]:
tandeo.to_file('../CLEAN/TABLEAU/tandeo2019.shp')

### INTENTANDO SACAR POBLACION POR COLONIA

Vale no esta bien este archivo

In [49]:
pob=pd.read_csv('../RAW/CENSOS INEGI/conjunto_de_datos_iter_09CSV20.csv')


In [52]:
pob['NOM_MUN'].value_counts()

NOM_MUN
Milpa Alta                              247
Tlalpan                                 167
Xochimilco                              114
Tláhuac                                  55
Cuajimalpa de Morelos                    27
La Magdalena Contreras                   26
Álvaro Obregón                            9
Total de la entidad Ciudad de México      3
Azcapotzalco                              2
Iztapalapa                                2
Iztacalco                                 2
Gustavo A. Madero                         2
Coyoacán                                  2
Benito Juárez                             2
Cuauhtémoc                                2
Miguel Hidalgo                            2
Venustiano Carranza                       2
Name: count, dtype: int64

### Limpieza archivo escuelas publicas

In [54]:
escuelas=pd.read_csv('../RAW/ESCUELAS/escuelas_publicas.csv', encoding='utf-8')

In [55]:
escuelas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2383 entries, 0 to 2382
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   nombre     2383 non-null   object 
 1   domicilio  2347 non-null   object 
 2   colonia    2231 non-null   object 
 3   alcaldia   2231 non-null   object 
 4   latitud    2242 non-null   float64
 5   longitud   2244 non-null   float64
dtypes: float64(2), object(4)
memory usage: 111.8+ KB


In [ ]:
#limpieza alcaldias

In [57]:
escuelas['alcaldia'] = escuelas['alcaldia'].str.title()

In [59]:
escuelas.loc[escuelas['alcaldia'] == 'Coyoacan', 'alcaldia'] = 'Coyoacán'
escuelas.loc[escuelas['alcaldia'] == 'Alvaro Obregon', 'alcaldia'] = 'Álvaro Obregón'
escuelas.loc[escuelas['alcaldia'] == 'Tlahuac', 'alcaldia'] = 'Tláhuac'
escuelas.loc[escuelas['alcaldia'] == 'Benito Juarez', 'alcaldia'] = 'Benito Juárez'
escuelas.loc[escuelas['alcaldia'] == 'Cuauhtemoc', 'alcaldia'] = 'Cuauhtémoc'
escuelas.loc[escuelas['alcaldia'] == 'Magdalena Contreras', 'alcaldia'] = 'La Magdalena Contreras'
escuelas.loc[escuelas['alcaldia'] == 'Cuajimalpa De Morelos', 'alcaldia'] = 'Cuajimalpa de Morelos'

Usaremos Shapely para sacar las colonias y sus IDS

In [61]:
geo=gpd.read_file('../CLEAN/TABLEAU/modified_geo.shp')

In [62]:
from shapely.geometry import Point, Polygon


for index, row in escuelas.iterrows():
     lat = row['latitud']
     lon = row['longitud']
    

     point = Point(lon, lat)
    

     colonia_name = None
    
 
     for _, polygon in geo.iterrows():

         if polygon['geometry'].contains(point):

             colonia_name = polygon['col_name']
             break 
    
   
     escuelas.at[index, 'Colonia name'] = colonia_name

In [65]:
nancols(escuelas)

colonia         152
alcaldia        152
Colonia name    146
latitud         141
longitud        139
domicilio        36
dtype: int64

In [66]:
escuelas.dropna(subset=['latitud'], inplace=True)


In [68]:
escuelas.dropna(subset=['colonia'], inplace=True)

In [69]:
nancols(escuelas)

domicilio    36
dtype: int64

In [71]:
escuelas['nombre'] = escuelas['nombre'].str.lower()

In [75]:
escuelas['nivel escolar'] = ''  

for index, row in escuelas.iterrows():
    nombre = row['nombre']
    if 'secundaria' in nombre:
        escuelas.at[index, 'nivel escolar'] = 'Secundaria'
    elif 'primaria' in nombre:
        escuelas.at[index, 'nivel escolar'] = 'Primaria'
    elif 'preescolar' in nombre:
        escuelas.at[index, 'nivel escolar'] = 'Preescolar'


In [84]:
nancols(escuelas)

domicilio    36
dtype: int64

In [85]:
col_unique_counts(escuelas, 'nivel escolar')

,nivel escolar,Count
0,Primaria,1134
1,Preescolar,643
2,Secundaria,444
3,,10


In [78]:
escuelas['nombre'] = escuelas['nombre'].str.title()
escuelas['domicilio'] = escuelas['domicilio'].str.title()
escuelas.drop(columns=['colonia'], inplace=True)


In [87]:
#escuelas

In [80]:
escuelas.to_csv('../CLEAN/TABLEAU/escuelas_publicas.csv')